In [1]:
import tensorflow as tf

class ConvBlock(tf.keras.layers.Layer):
    def __init__(self, filters, num_layers):
        super(ConvBlock, self).__init__()
        self.conv_layers = []
        self.bn_layers = []
        self.relu_layers = []

        for _ in range(num_layers):
            self.conv_layers.append(tf.keras.layers.Conv2D(filters, kernel_size=(3, 3), strides=(1, 1), padding='same'))
            self.bn_layers.append(tf.keras.layers.BatchNormalization())
            self.relu_layers.append(tf.keras.layers.ReLU())

        self.maxpool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))
        self.dropout = tf.keras.layers.Dropout(0.25)

    def call(self, inputs, training=False):
        x = inputs

        for conv, bn, relu in zip(self.conv_layers, self.bn_layers, self.relu_layers):
            x = conv(x)
            x = bn(x, training=training)
            x = relu(x)

        x = self.maxpool(x)
        x = self.dropout(x, training=training)
        return x


class MyVGG16(tf.keras.Model):
    def __init__(self, num_classes):
        super(MyVGG16, self).__init__()
        self.block1 = ConvBlock(64, num_layers=2)
        self.block2 = ConvBlock(128, num_layers=2)
        self.block3 = ConvBlock(256, num_layers=3)
        self.block4 = ConvBlock(512, num_layers=3)
        self.block5 = ConvBlock(512, num_layers=3)
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(4096, activation='relu')
        self.dropout1 = tf.keras.layers.Dropout(0.5)
        self.fc2 = tf.keras.layers.Dense(4096, activation='relu')
        self.dropout2 = tf.keras.layers.Dropout(0.5)
        self.fc3 = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs, training=False):
        x = self.block1(inputs, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.block4(x, training=training)
        x = self.block5(x, training=training)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout1(x, training=training)
        x = self.fc2(x)
        x = self.dropout2(x, training=training)
        x = self.fc3(x)
        return x




In [2]:
# Create an instance of the VGG16 model
num_classes = 7
model = MyVGG16(num_classes)

# Print the model summary
model.build(input_shape=(None, 224, 224, 3))
model.summary()

Model: "my_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_block (ConvBlock)      multiple                  39232     
                                                                 
 conv_block_1 (ConvBlock)    multiple                  222464    
                                                                 
 conv_block_2 (ConvBlock)    multiple                  1478400   
                                                                 
 conv_block_3 (ConvBlock)    multiple                  5905920   
                                                                 
 conv_block_4 (ConvBlock)    multiple                  7085568   
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  102